In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing.sequence import TimeseriesGenerator


data = pd.read_csv("/content/drive/My Drive/case_time_series_mod_dates.csv")
print(data.info())

dates= data['Date']
d  = pd.to_datetime(data['Date'])

data.set_axis(data['Date'], inplace= True)

last_dates = dates.values[-1]
print('last dates: ',dates.values[-1])
conf_data = data['Total Confirmed'].values
print(conf_data.shape)


In [0]:
# normalize the dataset  and making training and validation data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
conf_data = conf_data.reshape((-1,1))

conf_data = scaler.fit_transform(conf_data)
# split into train and test sets
train_size = int(len(conf_data) * 0.67)
test_size = len(conf_data) - train_size
train, test = conf_data[0:train_size,:], conf_data[train_size:len(conf_data),:]


split_perecent = 0.67
split = int(split_perecent *len(conf_data) )

dates_train = dates[:split]

dates_test = dates[split:]

In [0]:
#  As Keras LSTM expect input shoud be in the form of [samples, time_steps, features]
look_back = 5

train_data_gen = TimeseriesGenerator(train, train,
	length=look_back, sampling_rate=1,stride=1,
    batch_size=3)
test_data_gen = TimeseriesGenerator(test, test,
	length=look_back, sampling_rate=1,stride=1,
	batch_size=1)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

num_epochs = 30
model = Sequential()

model.add(
    LSTM(22,
         activation= 'relu',
         return_sequences= True,
         go_backwards=True,
         input_shape=(look_back,1))
)

model.add(LSTM(22, activation='relu'))

model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',  metrics=['accuracy'])

model.fit_generator(train_data_gen, epochs= 30, verbose=1)


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       
drive = GoogleDrive(gauth)

model.save('covid_19_15_1th_1.h5')
model_file = drive.CreateFile({'title' : 'models.h5'})                       
model_file.SetContentFile('models.h5')                       
model_file.Upload()

In [0]:
# load model 
from keras.models import load_model

model = load_model('/content/drive/My Drive/covid_19_15_1th_1.h5')


In [0]:
# predict on validation data

import plotly.graph_objects as go

prediction = model.predict_generator(test_data_gen)    

train = train.reshape((-1))   
'''
# (-1) converts (3*4) to 12 [1,2,3,4,5,...]
#(-1,1) converts ([[1,2,3,4]]) to ([[1],
                                    [2]...])
'''                   
test = test.reshape((-1))
prediction = prediction.reshape((-1))
print('prediction: ', prediction)

trace1 = go.Scatter(
    x= dates_train,
    y = train,
    mode = 'lines',
    name = "Training Data" 
)

trace2 = go.Scatter(
    x= dates_test,
    y= prediction,
    mode = 'lines',
    name = "Predictions"
)

trace3 = go.Scatter(
    x = dates_test,
    y = test,
    mode='lines',
    name = 'Ground Truth'    
)

layout = go.Layout(
    title = "Cases",
    xaxis= {'title': 'Date'},
    yaxis = { 'title' : 'Close'}
)

fig= go.Figure(data=[trace1, trace2, trace3], layout= layout)

fig.show()

In [0]:
# Forcast

conf_data = conf_data.reshape((-1))
look_back = 5

def predict(num_prediction, model):   # num_prediction = no. of days, a =  model
    prediction_list = conf_data[-look_back:]
    #print(prediction_list)

    for _ in range(num_prediction):     #  underscore(_) as a variable in looping.
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back,1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)

    prediction_list =  prediction_list[look_back-1:]

    return prediction_list
  
def predict_dates(num_prediction):
    last_date =  last_dates
  
    print('last date', last_date)
    prediction_dates = pd.date_range(last_date, periods= num_prediction +1).tolist() 
    print(prediction_dates)
    return prediction_dates

num_prediction = 7
forcast = predict(num_prediction, model)
forcast_dates = predict_dates(num_prediction)

In [19]:
import plotly.graph_objects as go

trace1 = go.Scatter(
    x = forcast_dates,
    y = forcast,
    mode = 'lines',
    name = 'Prediction'    
)

layout = go.Layout(
    title = "Covid-19 Confirmed Cases",
    xaxis= {'title': 'Date'},
    yaxis = { 'title' : 'Cases'}

)

fig = go.Figure(data=trace1, layout= layout)

fig.show()
